#Tesla Stock Monthly Predictions - By David Woo

## Data Import and Cleaning

In [1]:
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

In [2]:
import pandas as pd
import plotly.express as px

data = pd.read_csv('TSLA.csv')
data

,Date,Close/Last,Volume,Open,High,Low
0,11/25/2014,16.54,47320626,16.49,16.65,16.41
1,11/26/2014,16.56,29712465,16.56,16.60,16.44
2,11/28/2014,16.30,31795304,16.36,16.45,16.17
3,12/1/2014,15.44,129056920,16.08,16.16,15.27
4,12/2/2014,15.43,88073341,15.64,15.66,15.20
...,...,...,...,...,...,...
2512,11/19/2024,346.00,88852450,335.76,347.38,332.75
2513,11/20/2024,342.03,66340650,345.00,346.60,334.30
2514,11/21/2024,339.64,58011720,343.81,347.99,335.28
2515,11/22/2024,352.56,89140720,341.09,361.53,337.70


In [3]:
data.drop(['Volume', 'Open', 'High', 'Low'], axis=1, inplace = True)

data

,Date,Close/Last
0,11/25/2014,16.54
1,11/26/2014,16.56
2,11/28/2014,16.30
3,12/1/2014,15.44
4,12/2/2014,15.43
...,...,...
2512,11/19/2024,346.00
2513,11/20/2024,342.03
2514,11/21/2024,339.64
2515,11/22/2024,352.56


In [4]:
fig = px.line(data, x = 'Date', y = 'Close/Last', title='Tesla Monthly Stock Over Time')

fig.show()



##Data Prep for testing

In [5]:
def create_rolling_window_df(data, window_size):
    """Generate a DataFrame with rolling windows of closing prices."""
    columns = [f'N-{window_size-i}' for i in range(window_size)] + ['N']
    df = pd.DataFrame(columns=['Target Date'] + columns)

    for i in range(len(data) - window_size):
        row = [data.index[i + window_size]] + list(data[i:i + window_size + 1])
        df.loc[i] = row

    return df

window_size = 5
data_with_dates = data.set_index('Date')['Close/Last']
window_df = create_rolling_window_df(data_with_dates, window_size)

window_df

,Target Date,N-5,N-4,N-3,N-2,N-1,N
0,12/3/2014,16.54,16.56,16.30,15.44,15.43,15.29
1,12/4/2014,16.56,16.30,15.44,15.43,15.29,15.22
2,12/5/2014,16.30,15.44,15.43,15.29,15.22,14.91
3,12/8/2014,15.44,15.43,15.29,15.22,14.91,14.29
4,12/9/2014,15.43,15.29,15.22,14.91,14.29,14.46
...,...,...,...,...,...,...,...
2507,11/19/2024,328.49,330.24,311.18,320.72,338.74,346.00
2508,11/20/2024,330.24,311.18,320.72,338.74,346.00,342.03
2509,11/21/2024,311.18,320.72,338.74,346.00,342.03,339.64
2510,11/22/2024,320.72,338.74,346.00,342.03,339.64,352.56


In [6]:
x = window_df.iloc[:,1:-1].values
y = window_df['N'].values

x.shape, y.shape

((2512, 5), (2512,))

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, shuffle = False)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2009, 5), (503, 5), (2009,), (503,))

In [8]:

from sklearn.ensemble import RandomForestRegressor

# Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=5000, random_state=42, max_depth = 5)
rf_model.fit(x_train, y_train)

# Output feature importances (optional)
rf_model.feature_importances_


array([0.00269508, 0.00139715, 0.00374923, 0.02949541, 0.96266313])

In [9]:
# Predict on training and test sets
y_train_pred = rf_model.predict(x_train)
y_test_pred = rf_model.predict(x_test)


In [10]:
# Prepare training data for visualization
train_dates = window_df['Target Date'].iloc[:-len(y_test)].reset_index(drop=True)
train_results = pd.DataFrame({'Date': train_dates, 'Actual': y_train, 'Predicted': y_train_pred})

# Plot training set predictions vs actual values
fig = px.line(train_results, x='Date', y=['Actual', 'Predicted'], title='Actual vs Predicted on Training Data')
fig.show()



In [11]:
# Prepare test data for visualization
test_dates = window_df['Target Date'].iloc[-len(y_test):].reset_index(drop=True)
test_results = pd.DataFrame({'Date': test_dates, 'Actual': y_test, 'Predicted': y_test_pred})

# Plot test set predictions vs actual values
fig = px.line(test_results, x='Date', y=['Actual', 'Predicted'], title='Actual vs Predicted on Test Data')
fig.show()


In [12]:
from sklearn.metrics import mean_absolute_error

# Predict on the test set
y_test_pred = rf_model.predict(x_test)

# Calculate MAE for the test set
mae_test = mean_absolute_error(y_test, y_test_pred)
print(f"Test Set Mean Absolute Error: {mae_test:.2f}")

# Predict on the training set
y_train_pred = rf_model.predict(x_train)

# Calculate MAE for the training set
mae_train = mean_absolute_error(y_train, y_train_pred)
print(f"Training Set Mean Absolute Error: {mae_train:.2f}")


Test Set Mean Absolute Error: 6.67
Training Set Mean Absolute Error: 2.44


In [13]:

def recursive_forecast(model, initial_data, steps):
    """Predict future values using model recursively."""
    predictions = []
    input_data = list(initial_data)
    for _ in range(steps):
        prediction = model.predict([input_data[-window_size:]])[0]
        predictions.append(prediction)
        input_data.append(prediction)
    return predictions

# Predict the next 12 months
future_predictions = recursive_forecast(rf_model, data['Close/Last'].values[-window_size:], 12)

# Visualize the future predictions
future_dates = pd.date_range(data['Date'].iloc[-1], periods=13, freq='M').tolist()[1:]  # starts from the month after the last date in our data
future_data = pd.DataFrame({'Date': future_dates, 'Predicted': future_predictions})

fig = px.line(future_data, x='Date', y='Predicted', title='Predicted Tesla Stock Price for the Next 12 Months')
fig.show()

<ipython-input-13-ded3b67fc016>:15: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



[NbConvertApp] WARNING | pattern 'book.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execut